# Item 14: Prefer Exceptions To Returning None

In the case of dividing by zero, returning None seems natural because the result is undefined.

In [1]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

You may accidentally look for any `False` equivalent value to indicate errors instead of only looking for `None`.

In [2]:
x, y = 0, 5
result = divide(x, y)
if not result:
    print('Invalid inputs')  # This is wrong!

Invalid inputs


There are two ways to reduce the chance of such errors.
The first way is to split the return value into a two-tuple. 

In [3]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None


The second, better way to reduce these errors is to never return None at all. Instead, raise an exception up to the caller and make them deal with it. Here, I turn a ZeroDivisionError into a ValueError to indicate to the caller that the input values are bad:

In [4]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

# Item 15: Know How Closures Interact With Variable Scope

# Item 16: Consider Generators Instead Of Returning Lists

# Item 17: Be Defensive When Iterating Over Arguments

# Item 18: Reduce Visual Noise With Variable Positional Arguments

# Item 19: Provide Optional Behavior With Keyword Arguments

# Item 20: Use None And Docstrings To Specify Dynamic Default Arguments


# Item 21: Enforce Clarity With Keyword-Only Arguments